# Sending Text Messages based on Tweets

The goal of this tutorial is to introduce how to:
- get tweets from specific twitter handles.
- search tweets for relevant information.
- send messages based on contents of the tweets. 

This tutorial is based on code from [NYCParking](https://developer.twitter.com/en/docs/tutorials/nyc-parking) and [ExtractingTweetsTweepy](https://fairyonice.github.io/extract-someones-tweet-using-tweepy.html)


## NYC Parking Problem

- In New York City cars need to be moved most of the days for road cleaning, snow removal, etc; the rules are relaxed on a few days. 
- This notebook helps optimize when to move your car so that you do not move it when rules are relaxed, this is done by  sending text messages to your phone based on the contents of tweets from @NYCASP handle. 
- The @NYCASP twitter handle periodically tweets if parking rules - to move your car - are enforced on a particular day; failure to move your car when rules are in effect can leave you with a hefty parking ticket. 
- On a few days parking rules are suspended, it is unnecessary to move your car then, we want to send messages on days when the parking rules are suspended so that we don't have to move car unnecessarily.

## Instructions

- To be able to access tweets through twitter API, you will need to create a developer account - [TwitterDeveloperAccount](https://developer.twitter.com/en/docs/developer-portal/overview) 
- Creating a developer account grants you essential access. You will need to apply for Elevated or Academic/Research access to be able to run this ipython notebook; access to elevated or academic research may take a while to go through as there is a mandatory review process.
- To be able to send text messages to your phone you will need a twilio account. You can get a free twilio account by following instructions here - [Twilio](https://www.twilio.com/docs/usage/tutorials/how-to-use-your-free-trial-account)

## Download Required Packages

In [1]:
!pip install pandas
!pip install twilio
!pip install searchtweets

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Environment Setup

- The next step would be to set up environment variables for your twilio credentials. 
- Your twilio credentials will be available to you in your twilio account. 
- For cell phone number, enter the phone number on which you want to get the text messages with area code followed by the number.
- A secure way to use your credentials is by creating environment variables in your terminal. 
```console
export 'TWILIO_ACCOUNT_SID'='xxxxxxxxxxxxxxxxxxx'
export 'TWILIO_AUTH_TOKEN'='xxxxxxxxxxxxxxxxxxxxxxx'
export 'TWILIO_PHONE_NUMBER'='15555555555'
export 'CELL_PHONE_NUMBER'='16666666666'
```
- Set your twitter consumer_key, consumer_secret,  access_token, and access_token_secret as environment variables. 
- For information on where to locate this information you can look at [TwitterEnvironmentVariables](https://developer.twitter.com/en/docs/apps/overview)
```console
export 'consumer_key'='xxxx'
export 'consumer_secret'='xxxx'
export 'access_token'='xxxx'
export 'access_token_secret'='xxxx'
```
- After the authentication process, you will be able to access the twitter api interface.

## Tweepy Library

- To access the twitter API conveniently we use the tweepy library.
- For more details on this library check out [Tweepy](https://docs.tweepy.org/en/stable/)

## Import Required Modules

In [2]:
import os
from twilio.rest import Client
import datetime
import pandas as pd
import tweepy
from searchtweets import ResultStream, gen_rule_payload, load_credentials

## Getting Tweets From a Twitter Handle

- We will be using the tweepy library to access the twitter API. The twitter environement variables set up earlier will be used for [outh](https://oauth.net/2/) Authorization.
- user_id is the twitter handle of the user whose tweets you want to parse and take further action. 
- api.user_timeline gets the most recent 200 tweets from @NYCASP. Tweets contain the actual tweet and some metadata.
- For our use case, we would need to know:
    - when the tweet was generated: we want to act on the current day's tweet.
    - search for strings in the full text: we want to know when the rules are suspended.


In [3]:
user_id = "NYCASP"

consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

tweets = api.user_timeline(screen_name=user_id, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

print(tweets[1])
print("\n")
print("printing out the latest 2 tweets:")
print("\n")
for info in tweets[:2]:
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     print("\n")

Status(_api=<tweepy.api.API object at 0x7ff5ee216898>, _json={'created_at': 'Tue Mar 29 20:01:14 +0000 2022', 'id': 1508897087764717574, 'id_str': '1508897087764717574', 'full_text': '#NYCASP rules will be in effect tomorrow, Wednesday, March 30.', 'truncated': False, 'display_text_range': [0, 62], 'entities': {'hashtags': [{'text': 'NYCASP', 'indices': [0, 7]}], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 102773464, 'id_str': '102773464', 'name': 'NYC Alt Side Parking', 'screen_name': 'NYCASP', 'location': 'New York City', 'description': 'ASP info twice daily & for emergency suspensions.\n\nNew residential rules: https://t.co/I10feSs8iC\n\nAccount automated, will not respond. Use Policy: https://t.co/gsR1VH3Jzp', 'url': 'http

## Pandas Dataframe 

- Pandas is a library that allows easy and convenient manipulation of data. 
- For this problem it is much easier to search through the text if the raw tweets and the date created are converted into a Pandas dataframe. 
- For more information on Pandas checkout [Pandas](https://pandas.pydata.org/)


In [4]:
tweet_text = []
tweet_date = []
for info in tweets:
    tweet_text.append(info.full_text)
    tweet_date.append(info.created_at)
    
df = pd.DataFrame({'tweet':tweet_text, 'date':tweet_date})

df.head()

,tweet,date
0,"#NYCASP rules are in effect today, March 30",2022-03-30 11:30:13+00:00
1,"#NYCASP rules will be in effect tomorrow, Wedn...",2022-03-29 20:01:14+00:00
2,"#NYCASP rules are in effect today, March 29",2022-03-29 11:30:12+00:00
3,"#NYCASP rules will be in effect tomorrow, Tues...",2022-03-28 20:01:13+00:00
4,"#NYCASP rules are in effect today, March 28.",2022-03-28 11:30:13+00:00


## Twilio Client and Send Message API

- twilio_connect initializes the twilio client for accessing API to send text message.
- send_message sends the message in the variable body - which is customizable - to the cell phone number specified by the environment variable CELL_PHONE_NUMBER

In [5]:
def twilio_connect():
    account_sid = os.environ.get('TWILIO_ACCOUNT_SID')
    auth_token = os.environ.get('TWILIO_AUTH_TOKEN')
    client = Client(account_sid, auth_token)
    return client

def send_message(client):
    return client.messages.create(from_=os.environ.get('TWILIO_PHONE_NUMBER'),
    to=os.environ.get('CELL_PHONE_NUMBER'),
    body="You don't have to move your car tonight. Enjoy your night!")

client = twilio_connect()

## Send Text Through Twilio

- Send a text message to your cell phone to remind that you need not move your car if "suspended" or "tomorrow" is in the body of the latest tweet. 
- If you do not get a text message then you need to move your car.
- Note that this code can be easily changed to send a text message if you need to move the car instead.

In [6]:
if 'suspended' in df['tweet'].values[0]:
    if 'tomorrow' in df['tweet'].values[0]:
        send_message(client=client)
        print('text sent')
    else:
        print('suspended but not tomorrow, no text sent')
else:
    print('not suspended, no text sent')

not suspended, no text sent
